In [47]:
# https://plot.ly/python/subplots/
import plotly as py
import plotly.graph_objs as go
# from plotly.offline import download_plotlyjs

import scipy

py.offline.init_notebook_mode(connected=True)

import numpy as np

import math

def lcm(a,b):
    return abs(a * b) // math.gcd(a,b) if a and b else 0


In [88]:
def interpolate(signal, time, m): # frequency *m
    signal_ = []
    time_ = []
    dt = (time[1] - time[0]) / m
    
    for i in range(len(signal) - 1):
        signal_.append(signal[i])
        signal_.extend([0] * (m - 1))
        
        time_.extend([time[i] + dt * j for j in range(m)])
    
    signal_.append(signal[-1])
    time_.append(time[-1])
    
    return signal_, time_

def decimate(signal, time, n): # frequency /m
    signal_ = []
    time_ = []
    
    for i in range(0, len(signal), n):
        signal_.append(signal[i])
        time_.append(time[i])
    
    return signal_, time_

def butter_lowpass(cutoff, fs, order=5):
    normal_cutoff = cutoff / fs
    b, a = scipy.signal.butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = scipy.signal.filtfilt(b, a, data)
    return y

def resample(s, t, f1, f2):
    m = lcm(f1, f2)
    order = 5
    si, ti = interpolate(s, t, m // f1)
    sf = butter_lowpass_filter(si, f1, m, order)
    sd, td = decimate(sf, ti, m // f2)

    return sd, td


In [89]:
N = 25
T = 1#s

f1 = N // T       # sample rate, Hz
f2 = f1 * 2

f = 1
w = 2 * np.pi * f

t = np.linspace(0, T, N)

s = [np.sin(w * i) for i in t]

sr, tr = resample(s, t, f1, f2)

m = f2 // f1
sr = [i * m for i in sr]

tr_s = go.Scatter(
    x = t,
    y = s,
    name='signal',
    mode='markers'
)

tr_sr = go.Scatter(
    x = tr,
    y = sr,
    name='resampled',
    mode='markers'
)

rows = 1
cols = 1

fig = py.tools.make_subplots(rows = rows, cols = cols)
fig['layout'].update(height=rows * 450, width=cols * 900)

fig.append_trace(tr_s, 1, 1)
fig.append_trace(tr_sr, 1, 1)

py.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]

